# VacationPy

### Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,55.76,84,4,20.98,NZ,1699919128
1,1,vorkuta,67.5000,64.0000,3.72,93,95,6.60,RU,1699919721
2,2,mejit,10.2753,170.8646,82.36,81,68,17.94,MH,1699919682
3,3,qamdo,31.1667,97.2333,20.10,58,3,4.09,CN,1699919721
4,4,port-aux-francais,-49.3500,70.2167,44.35,97,100,37.22,TF,1699919722


### Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [8]:
# Configure the map plot_1
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "Humidity",
    cmap = "viridis"
)

# Display the map plot_1
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                   & (city_data_df["Max Temp"] > 21)
                                   & (city_data_df["Wind Speed"] < 4.5)
                                   & (city_data_df["Cloudiness"] > 90)]
                                   
 
# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,qaqortoq,60.7167,-46.0333,21.31,93,100,2.82,GL,1699919734
287,287,badger,64.8000,-147.5333,26.98,93,100,1.99,US,1699919683
398,398,ust-tsilma,65.4410,52.1498,22.60,95,100,3.00,RU,1699919869
425,425,tulun,54.5611,100.5792,23.11,84,100,4.32,RU,1699919880


### Step 3: Create a new DataFrame called hotel_df.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
34,qaqortoq,GL,60.7167,-46.0333,93,
287,badger,US,64.8000,-147.5333,93,
398,ust-tsilma,RU,65.4410,52.1498,95,
425,tulun,RU,54.5611,100.5792,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
          }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity:{longitude}, {latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
qaqortoq - nearest hotel: No hotel found
badger - nearest hotel: No hotel found
ust-tsilma - nearest hotel: No hotel found
tulun - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
34,qaqortoq,GL,60.7167,-46.0333,93,No hotel found
287,badger,US,64.8000,-147.5333,93,No hotel found
398,ust-tsilma,RU,65.4410,52.1498,95,No hotel found
425,tulun,RU,54.5611,100.5792,84,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", "Lat", 
                                   geo = True, 
                                   tiles="OSM",
                                   color = "City",
                                   hover_cols = ["Hotel Name","Country"])

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)